In [16]:
import pandas
filename = 'results-survey976727_pre.csv'
data = pandas.read_csv(filename, sep=';', header=0)

data.drop(['submitdate', 'G00Q01[SQ01]', 'G02Q01', 'G02Q02', 'G02Q03', 'G02Q04', 'G03Q02[other]'], axis = 'columns', inplace=True)data


,id,token,G03Q01,G03Q02
0,21,gqoj,99,Masculino
1,22,hgfx,99,Masculino
2,23,gton,99,Masculino
3,24,uala,12,Masculino
4,25,qeqyj,13,Masculino
...,...,...,...,...
141,162,cojqb,8,Femenino
142,163,rfofs,13,Masculino
143,164,fmupf,12,Masculino
144,165,olugf,12,Masculino


In [18]:
# Calcular los límites de los valores atípicos
# regla empírica, que establece que los valores atípicos se encuentran a más de 3 desviaciones estándar de la media.

edad_media = data['G03Q01'].mean()
edad_desv_est = data['G03Q01'].std()
edad_limite_inferior = edad_media - (3*edad_desv_est)
edad_limite_superior = edad_media + (3*edad_desv_est)

# Filtrar el dataframe para eliminar los outliers de edad
data_filtrado = data.loc[(data['G03Q01'] >= edad_limite_inferior) & (data['G03Q01'] <= edad_limite_superior)]

data_filtrado

,id,token,G03Q01,G03Q02
3,24,uala,12,Masculino
4,25,qeqyj,13,Masculino
5,26,eprdu,14,Masculino
6,27,fkmtp,13,Masculino
7,28,pbgky,13,Masculino
...,...,...,...,...
141,162,cojqb,8,Femenino
142,163,rfofs,13,Masculino
143,164,fmupf,12,Masculino
144,165,olugf,12,Masculino
